In [7]:
%run -i '/Users/dilrubareyyan/Desktop/Turkish-Lemmatizer/trainLexicon.py'

Negative verbs are added.
Zero infinitive forms of verbs are added.
Consonant softening forms are added.
Dropping vowel forms are added.
Becoming close vowel forms are added.
Transformed lexicon is saved to revisedDict.pkl


# Lemmatizer from @akoksal

In [15]:
import re
import json
import pickle
import sys

class Lemmatizer:
    def __init__(self):
        try:
            with open('revisedDict.pkl', 'rb') as f:
                self.revisedDict = pickle.load(f)
        except IOError:
            print("Please run trainLexicon.py to generate revisedDict.pkl file")

    def check(self, root, suffix, guess, action):
        if action == "unsuz yumusamasi":
            return len(suffix)>0 and suffix[0] in ["a","e","ı","i","o","ö","u","ü"] and self.checkSuffixValidation(suffix)[0]
        if action == "unlu daralmasi":
            if guess=="demek" and self.checkSuffixValidation(suffix)[0]:
                return True
            if guess=="yemek" and self.checkSuffixValidation(suffix)[0]:
                return True
            
            if suffix.startswith("yor"):
                lastVowel = ""
                for letter in reversed(guess[:-3]):
                    if letter in ["a","e","ı","i","o","ö","u","ü"]:
                        lastVowel = letter
                        break
                if lastVowel in ["a","e"] and self.checkSuffixValidation(suffix)[0]:
                    return True
            return False
        if action == "fiil" or action == "olumsuzluk eki":
            return self.checkSuffixValidation(suffix)[0] and not ((root.endswith("la") or (root.endswith("le"))) and suffix.startswith("r"))
        if action == "unlu dusmesi":
            count = 0
            for letter in guess:
                if letter in ["a","e","ı","i","o","ö","u","ü"]:
                    count+=1
                    lastVowel = letter
            if self.checkSuffixValidation(suffix)[0] and count==2 and (lastVowel in ["ı","i","u","ü"]) and (len(suffix)>0 and suffix[0] in ["a","e","ı","i","o","ö","u","ü"]):
                if lastVowel == "ı":
                    return suffix[0] in ["a","ı"]
                elif lastVowel == "i":
                    return suffix[0] in ["e","i"]
                elif lastVowel == "u":
                    return suffix[0] in ["a","u"]
                elif lastVowel == "ü":
                    return suffix[0] in ["e","ü"]
            return False
        return True
        
    def findPos(self, kelime):
        revisedDict = self.revisedDict
        l = []
        if "'" in kelime:
            l.append([kelime[:kelime.index("'")]+"_1","tirnaksiz",kelime])
        mid = []
        for i in range(len(kelime)):
            guess = kelime[:len(kelime)-i]
            suffix = kelime[len(kelime)-i:]
            ct = 1
            
            while guess+"_"+str(ct) in revisedDict:
                if self.check(guess, suffix, revisedDict[guess+"_"+str(ct)][1], revisedDict[guess+"_"+str(ct)][0]):
                    guessList = (revisedDict[guess+"_"+str(ct)])
                    while guessList[0] not in ["kok","fiil","olumsuzluk"]:
                        guessList = revisedDict[guessList[1]]
                    mid.append([guessList[1], revisedDict[guess+"_"+str(ct)][0],guess+"_"+str(ct)])
                ct = ct+1
                
        temp = []
        for kel in mid:
            kelime_kok = kel[0][:kel[0].index("_")]
            kelime_len = len(kelime_kok)
            if kelime_kok.endswith("mak") or kelime_kok.endswith("mek"):
                kelime_len -= 3
            not_inserted = True
            for index in range(len(temp)):
                temp_kelime = temp[index]
                temp_kelime_kok = temp_kelime[0][:temp_kelime[0].index("_")]
                temp_len = len(temp_kelime_kok)
                if temp_kelime_kok.endswith("mak") or temp_kelime_kok.endswith("mek"):
                    temp_len -= 3
                if(kelime_len>temp_len):
                    temp.insert(index,kel)
                    not_inserted = False
            if not_inserted:
                temp.append(kel)
        output = l+temp
        if len(output)==0:
            output.append([kelime+"_1","çaresiz",kelime+"_1",])
        return output

    def checkSuffixValidation(self, suff):
        suffixList = ["","a", "abil", "acağ", "acak", "alım", "ama", "an", "ar", "arak", "asın", "asınız", "ayım", "da", "dan", "de", "den", "dı", "dığ", "dık", "dıkça", "dır", "di", "diğ", "dik", "dikçe", "dir", "du", "duğ", "duk", "dukça", "dur", "dü", "düğ", "dük", "dükçe", "dür", "e", "ebil", "eceğ", "ecek", "elim", "eme", "en", "er", "erek", "esin", "esiniz", "eyim", "ı", "ıl", "ım", "ımız", "ın", "ınca", "ınız", "ıp", "ır", "ıyor", "ız", "i", "il", "im", "imiz", "in", "ince", "iniz", "ip", "ir", "iyor", "iz", "k", "ken", "la", "lar", "ları", "ların", "le", "ler", "leri", "lerin", "m", "ma", "madan", "mak", "maksızın", "makta", "maktansa", "malı", "maz", "me", "meden", "mek", "meksizin", "mekte", "mektense", "meli", "mez", "mı", "mış", "mız", "mi", "miş", "miz", "mu", "muş", "mü", "muz", "müş", "müz", "n", "nın", "nız", "nin", "niz", "nun", "nuz", "nün", "nüz", "r", "sa", "se", "sı", "sın", "sınız", "sınlar", "si", "sin", "siniz", "sinler", "su", "sun", "sunlar", "sunuz", "sü", "sün", "sünler", "sünüz", "ta", "tan", "te", "ten", "tı", "tığ", "tık", "tıkça", "tır", "ti", "tiğ", "tik", "tikçe", "tir", "tu", "tuğ", "tuk", "tukça", "tur", "tü", "tüğ", "tük", "tükçe", "tür", "u", "ul", "um", "umuz", "un", "unca", "unuz", "up", "ur", "uyor", "uz", "ü", "ül", "ün", "üm", "ümüz", "ünce", "ünüz", "üp", "ür", "üyor", "üz", "ya", "yabil", "yacağ", "yacak", "yalım", "yama", "yan", "yarak", "yasın", "yasınız", "yayım", "ydı", "ydi", "ydu", "ydü", "ye", "yebil", "yeceğ", "yecek", "yelim", "yeme", "yen", "yerek", "yesin", "yesiniz", "yeyim", "yı", "yım", "yın", "yınca", "yınız", "yıp", "yız", "yi", "yim", "yin", "yince", "yiniz", "yip", "yiz", "yken", "yla", "yle", "ymış", "ymiş", "ymuş", "ymüş", "yor", "ysa", "yse", "yu", "yum", "yun", "yunca", "yunuz", "yup", "yü", "yuz", "yüm", "yün", "yünce", "yünüz", "yüp", "yüz"]
        validList = []
        if suff in suffixList:
            validList.append(suff)
        for ind in range(1,len(suff)):
            if(suff[:ind] in suffixList):
                cont, contList = self.checkSuffixValidation(suff[ind:])
                if cont:
                    contList = [suff[:ind]+"+"+l for l in contList]
                    validList = validList+contList
        return len(validList)>0,validList

    def lemmatize(self, text):
        dc  = []
        for i in text.split(' '):
            findings = self.findPos(i.lower())
            word = findings[0][0]
            dc.append(findings[0][0][:-2])
    
        return dc

# Helper Functions

In [51]:
import re
from string import punctuation

def remove_punctuation(text):
    """
    Removes punctuations from given text
    """
    # Replace punctuation with space instead of remove it for hand-to-mouth, six-week-old, euro-certificate
    return text.translate(str.maketrans(punctuation, ' ' * len(punctuation)))


def remove_extra_whitespaces(text):
    """
    Removes extra whitespaces from given text such as multiple adjencent space
    """
    return re.sub(r'\s+', ' ', text).strip()


def normalize(text):
    """
    Normalizes the text.
    """
    return remove_extra_whitespaces(remove_punctuation(text))


# Test

In [49]:
test_text = "Demir çelik sanayisinin temel ham maddesi olan ve ülkemizde en çok tüketilen madenlerden biri demirdir. Demir rezervi bakımından en zengin yataklar, Sivas ve Malatya illerinde bulunmaktadır. Divriği, Hasançelebi ve Hekimhan, bu illerde demir yataklarının bulunduğu başlıca yerlerdir. Diğer önemli demir yatakları Niğde, Bingöl ve Kesikköprü’dedir (Ankara).Bu yerlerden çıkarılan demir Ereğli (Zonguldak), Karabük ve İskenderun demir çelik fabrikalarında işlenmektedir."
lemmatizer = Lemmatizer()
print(lemmatizer.lemmatize(normalize(test_text))) 


['demir', 'çelik', 'sanayi', 'temel', 'ham', 'madde', 'ol', 've', 'ülkem', 'en', 'çok', 'tüket', 'maden', 'biri', 'demir', 'demir', 'rezerv', 'bakım', 'en', 'zengin', 'yatak', 'sivas', 've', 'malatya', 'il', 'bulun', 'divriği', 'hasan', 've', 'hekimhan', 'bu', 'il', 'demir', 'yatak', 'bulun', 'başlıca', 'yer', 'diğer', 'önemli', 'demir', 'yatak', 'niğde', 'bingöl', 've', 'kesik', 'ankara', 'bu', 'yer', 'çıkar', 'demir', 'ereğli', 'zonguldak', 'karabük', 've', 'i', 'demir', 'çelik', 'fabrika', 'işle']


## Read Database

In [53]:
import os, re

'''
Transfer DERLEM to a Dictionary.
'''
with open("derlem.txt", encoding="utf-16") as f:
    opened_file = f.readlines()
   
'''
The dictionary of the DERLEM.
{id: paragraph as string}
'''
file_dict = {}
for i, line in enumerate(opened_file):
    if line != "\n":
        '''
        The first number in the line is the Paragraph ID,
        assign the Paragraph ID as the key to the dictionary,
        and the rest of the paragraph to the value.
        '''
        file_dict[int(line.split(' ', 1)[0])] = re.findall(" (.*)", line)[0]
    
# print(len(file_dict))

'''
The normalized dictionary of the DERLEM.
{id: array of tokens}
'''
norm_dict = file_dict.copy()
for i in norm_dict:
    norm_dict[i] = lemmatizer.lemmatize(normalize(file_dict[i]))
    
# print(norm_dict)